In [5]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
from pymongo import MongoClient


# https://tabelog.com/tw/tokyo/rstLst/"+ str(page) +"/?SrtT=rt
page = 1
while True:
    
    
    print("======================================================================================================")
    print("處理頁面",page)
    print("\n")
    try:
        url = "https://icook.tw/categories/63?page="+ str(page) +"/"
        headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:99.0) Gecko/20100101 Firefox/99.0"}
        response = requests.get(url, headers=headers)
        html = BeautifulSoup(response.text, 'html.parser')
        
        if response.status_code == 200:  #正確     
           
                
            food_name = html.find_all('h2', class_ = 'browse-recipe-name')
            l_fname = []
            for i in food_name:
                l_fname.append(i.getText().replace('\n',''))
            
#              print(food_name)
#             print(len(l_fname))
            
            food_ingredient = html.find_all('p', class_ = 'browse-recipe-content-ingredient')
            #print(food_ingredient.text)
            l_ingredient = []
            for i in food_ingredient:
                ingredient = i.getText()
                l_ingredient.append(ingredient.replace('\n','').replace('食材：','').split('、'))
            
#             print(l_ingredient)
            
            href = html.find_all('a', class_ = 'browse-recipe-link')
            #food_href = ("https://icook.tw" + str(href[0].get("href")))
            #print(food_href)
            l_href = []
            for i in href:
                ref = 'https://icook.tw'+i.get("href")
                l_href.append(ref)
            
            #print(l_href)
        
            img_list=[]
            all_imgs = html.find_all('img', class_ = 'browse-recipe-cover-img lazyload')
            #print(all_imgs)
            for i in all_imgs:
                img_src = i.get("data-src")
                #print(img_list)
                img_list.append(img_src)
            
            #print(img_list)
                
                      
            df = pd.DataFrame((zip(l_fname, l_ingredient, l_href, img_list)),columns=['食物名稱', '食材', '食譜網址', '料理圖片'])
            print(df)
            
            for i in df.index:
                myclient = MongoClient('mongodb://localhost:27017/')
                mydb = myclient["food"]
 
                mycol = mydb["義式料理"]
                
                mydict = { "food_name":df['食物名稱'][i], "food_ingredient":df['食材'][i] , "href": df['食譜網址'][i] , "imgs":df['料理圖片'][i]}
 
                x = mycol.insert_one(mydict) 
                print(x)
        
                collist = mydb. list_collection_names()
                # collist = mydb.collection_names()
                if "義式料理" in collist:   # 判断 recipe 集合是否存在
                    print("集合已存在！")                  
                
            page = page + 1   
        else:
            print('好像是最後一頁')
            break
        
    except requests.ConnectionError as conn_ex:
        print("連線錯誤")
        print(str(conn_ex))
        break
    
    

    
    






處理頁面 1


18
                   食物名稱                                                 食材  \
0                義式茄汁肉丸  [橄欖油, 奶油, 牛絞肉, 蒜末, 豬絞肉, 去皮番茄, 雞蛋, 鹽, 洋蔥丁, 黑胡椒,...   
1     鮮蛤番茄義大利麵佐西班牙茄汁麵包塔  [百味來義大利直麵, 百味來拿坡里蔬菜番茄義大利麵醬, 新鮮蛤蠣, 初榨橄欖油, 洋蔥切碎,...   
2   VIP 專屬【名菜自己做】義式獵人燉雞  [雞腿肉(切塊), 馬鈴薯, 月桂葉, 紅蘿蔔, 義式香料, 洋蔥, 鹽, 培根, 番茄糊,...   
3     Olivia♥義式小肉丸番茄蔬菜湯  [豬絞肉, 板豆腐, 新鮮香草或乾燥義式香料, 鹽巴, 黑胡椒, 洋蔥, 大蒜, 西洋芹, ...   
4             大蒜橄欖油義大利麵     [義大利麵, 大蒜, 橄欖油, 辣椒片, 鹽巴, Parsley 歐芹, 黑胡椒粉, 檸檬]   
5              南瓜干貝義大利麵  [小南瓜, 干貝, 義大利麵, 洋蔥, 培根, 蒜頭, 橄欖油, 鮮奶, 鮮奶油, 鹽, 黑...   
6        <總會有辦法的>墨魚義大利麵  [槍烏賊, 墨魚汁, 茄汁義大利麵醬, 紅蔥頭, 蒜, 鯷魚, 白酒, 橄欖油, 鹽, 番茄...   
7      【防疫料理】罐頭肉醬番茄義大利麵    [義大利麵, 胡蘿蔔, 黑胡椒粉, 洋蔥, 糖, 牛番茄, 鹽, 大蒜, 九層塔, 肉末罐頭]   
8        匈牙利紅椒優格雞胸肉義大利麵  [義大利麵, 雞胸肉, 匈牙利紅椒粉, 無糖優格, 辣椒粉, 安食小品醬鹽, 黑胡椒粒, 洋...   
9            青醬義大利麵（菠菜）  [義大利麵, 雞肉, 蒜末, 醬油, Spinach / 菠菜, 黑胡椒粉, Cream c...   
10           健康蔬食青醬義大利麵                      [酪梨, 義大利麵, 蘑菇, 青醬, 蒜, 小番茄, 鹽]   
11             檸檬鮭魚義大利麵  [鮭魚, 義大利麵, 芝麻葉, 洋蔥, 蒜頭, 鹽, 

18
                    食物名稱                                                 食材  \
0               蒜味雞肉義大利麵            [義大利麵, 雞胸肉, 德式香腸, 蒜頭, 味調小屋香蒜抹醬, 橄欖油, 鹽]   
1               三蔬鴨肉義大利麵            [櫻桃鴨肉絲, 洋蔥絲, 義大利香料, 紅蔥頭片, 鹽, 青蔥絲, 義大利麵]   
2     清炒花椰菜雞肉義大利麵（簡易家常菜）         [義大利天使麵, 雞胸肉, 花椰菜, 起司, 洋蔥, 鹽, 黑胡椒, 辣椒, 蒜頭]   
3     柚香肉燥義大利麵★省時義大利麵速煮法         [義大利麵, 豬里肌瘦絞肉, 牛番茄, 柚肉和風醬, 鹽/白胡椒粉/蒜粉, 橄欖油]   
4    [便當] 清炒蔬食筆管麵 (義大利麵)                              [筆管麵, 鴻喜菇, 青菜, 白酒, 油]   
5                芝士龍蝦焗麵🦞  [波士頓龍蝦, 洋蔥, 雞湯or高湯, 蒜頭, 鮮奶, 雞蛋, 青蔥, 奶油, 太白粉, 營...   
6               叉燒雞肉義大利麵  [義大利捲心麵, 雞蛋, 洋菇, 黃甜椒(切丁), 黑胡椒, 紅洋蔥(切丁)/白洋蔥也可, ...   
7   鮮蝦花枝通心麵 - 小朋友最愛的繽紛料理  [通心麵, 飲用水, 帶尾大蝦仁, 花枝, 豬肉絲, 玉米筍, 荷蘭豆, 紅蘿蔔, 德式香腸...   
8            [蔬食] 春天義大利麵  [小番茄, 牛番茄, 義大利短麵, Mozzarella 乳酪, 義大利瓜 Zucchini...   
9     Carbonara // 卡邦尼意粉                   [意粉, 淡忌廉, 牛奶, 洋蔥, 蘑菇, 煙肉, 巴馬臣芝士]   
10              抹茶起司義大利麵  [義大利麵, 夏威夷豆, 抹茶粉, Cream Cheese, 牛奶, 洋蔥, 蘑菇, 大蒜...   
11     影片◆簡易菠菜肉醬千層麵◆烤箱料理  [起司絲, 起司片, 牛絞肉, 千層麵, 洋蔥

18
                   食物名稱                                                 食材  \
0             檸檬荷蘭芹義大利麵         [義大利麵, 檸檬, 蒜, 荷蘭芹, 乾辣椒碎, 橄欖油, 巴馬干酪粉, 鹽和胡椒]   
1                義大利肉醬麵  [義大利麵條, 水, 過濾水, 鹽, 切塊雞肉, 豬絞肉（帶肥胛心肉）, 洋蔥, 洋蔥丁, ...   
2               起司pizza                    [中筋麵粉, 酵母, 起司, 暖水, 蜂蜜, 糖, 鹽, 油]   
3              番茄肉醬義大利麵         [番茄, 洋蔥, 蒜頭, 絞肉, 鹽巴, 月桂葉, 義大利香料, 黑胡椒, 番茄醬]   
4            洋蔥香腸起司吐司披薩              [厚吐司, 香腸, 蕃茄醬, 洋蔥碎, 香草/黑胡椒粉, Pizza起司]   
5              海鮮青醬義大利麵  [淡菜, 蛤蠣, 九層塔(青骨), 義大利麵, 蒜頭, 洋蔥, 帶皮花生, 橄欖油, 鹽巴,...   
6               義大利香料燒餅               [中筋麵粉, 水, 油, 鹽, 糖, 義大利香料, 生白芝麻, 酵母粉]   
7            義大利肉醬麵-寶寶版                              [牛番茄, 洋蔥, 蒜頭, 油, 豬絞肉]   
8              青蔬雞肉義大利麵  [義大利麵（直麵）, 橄欖油, 蒜頭, 洋蔥, 雞胸肉, 鴻禧菇（洋菇, 香菇等皆可）, 黑...   
9            義式培根蛋汁義大利麵                           [義大利麵, 蛋黃, 起司粉, 黑胡椒, 培根]   
10              明太子義大利麵  [義大利麵, 鹽巴, 明太子, 黑胡椒, 無鹽奶油(或鮮奶油), 洋蔥, 鰹魚醬油, 紅蔥頭...   
11   簡單的「夏威夷披薩吐司」可愛又好吃。                            [吐司, 番茄醬, 

18
                    食物名稱                                                 食材  \
0                經典義大利紅醬  [碎番茄罐頭, 黃洋蔥, 大蒜, 紅蘿蔔, 西芹, 鹽, 黑胡椒, 乾奧勒岡, 乾羅勒, 月...   
1               茄汁肉醬義大利麵       [義大利麵, 牛絞肉, 橄欖油, 鹽水, 番茄醬, 牛番茄, 黑胡椒, 九層塔, 蒜頭]   
2   [一鍋到底] 櫻花蝦義大利麵 (露營O)                    [義大利麵, 高麗菜, 櫻花蝦 (泡一下水), 蔥 (切段)]   
3      義式蕃茄冷麵【懶人料理・便當盒版】  [義大利麵, 橄欖油, 牛番茄, 鹽巴, 小黃瓜, 檸檬, 洋蔥, 糖, 蒜頭, Tabas...   
4               清炒海鮮義大利麵  [蝦子, 小捲（或花枝）, 甜椒（黃紅）, 美白菇, 義大利麵（粗細）, 洋蔥絲, 黑胡椒粒...   
5              🥣懶人早餐-披薩餡     [起司（必備）, 番茄醬（必備）, 蛋（必備）, 吐司（必備）, 生菜, 火腿, 鮪魚罐頭]   
6              焗烤茄汁蔬菜千層麵  [自製紅醬, 鷹嘴豆罐頭, 杏鮑菇, 素火腿, 紅甜椒, 黃甜椒, 茄子, 櫛瓜, 千層麵,...   
7             白酱奶油玉米义大利面  [义大利面, 绞肉（或培根也可以）, 鸿喜菇, 红萝卜, 玉米粒, 乳玛琳, 蒜头, 北海道...   
8        巴沙米克醋炒野菇（萬用常備菜）  [鮮香菇（中型）, 杏鮑菇（中型）, 巴沙米克醋, 鴻禧菇, 鹽, 蒜頭, 黑胡椒, 橄欖油...   
9                 提拉米蘇蛋糕     [馬斯卡彭乳酪, 動物性鮮奶油, 蛋黃, 砂糖, 吉利丁片, 濃縮咖啡, 咖啡酒, 可可粉]   
10                義式涼拌南瓜  [南瓜　, 蒜末, 紅辣椒末, 糯米醋, 紅葡萄酒　（也可以用米酒！）, 烹大師或鹽巴, 冰...   
11         簡單0失敗【白醬意大利麵】     [新鮮洋蔥, 全聯新鮮蝦仁, 意大利麵,

18
                    食物名稱                                                 食材  \
0                 義大利肉醬麵  [豬絞肉(細), 小顆牛番茄, 番茄醬, 小顆洋蔥, 黑胡椒(粗粒）, 大蒜, 鹽, 起司片...   
1           千層麵連義大利人都說好吃  [千層麵, 莫札瑞拉起士絲, 無鹽奶油, 帕瑪森起士粉, 低筋麵粉, 牛絞肉, 橄欖油, 牛...   
2                 豬肉炒通心麵  [通心麵, 豬肉片, 醬油, 紅蘿蔔, 香菇素蠔油, 鹽, 鴻喜菇, 烏醋, 糖, 蒜頭, ...   
3     氣炸雞腿排(只要4步驟，超輕鬆享受)             [雞腿排, 蒜頭, 橄欖油, 甜椒, 義式香料, 小番茄, 鹽巴, 黑胡椒]   
4                  肉醬千層麵  [豬絞肉, 洋蔥, 橄欖油, 番茄, 番茄醬, 蒜頭, 月桂葉, 紅蘿蔔, 百里香, 高湯,...   
5              自製免發酵薄片披薩  [奶油, 糖, 喜歡的蔬菜（玉米筍, 菇菇）, 鹽, 番茄醬, 中筋麵粉, 蝦仁可以替換自己...   
6   義式檸檬香雞排CHICKEN PICCA  [雞胸, 奶油, 橄欖油, 白酒, 檸檬汁, 大蒜, 紅蔥頭, 酸豆, 麵粉, 義大利香料,...   
7    香辣番茄肉醬義大利麵。一鍋到底義大利麵   [豬瘦絞肉, 牛番茄, 義大利麵, 水, 香辣牛肉醬(牛肉辣椒醬), 番茄醬, 鹽, 黑胡椒粒]   
8          夏日晚餐-櫛瓜雞肉奶油意麵                  [櫛瓜, 中型番茄, 洋蔥, 雞里肌, 義大利麵, 牛奶, 奶油]   
9         日式喫茶店 茄汁香腸義大利麵    [義大利麵, 小香腸, 青椒, 洋蔥, 紅椒, 橄欖油, 鹽, 水, 番茄醬, 醬油, 蠔油]   
10                橄欖油漬番茄                        [牛番茄（大顆）, 蒜片, 義式香料, 鹽, 橄欖油]   
11           蛤蠣蒜味橄欖油義大利麵                   [義大利麵,

...

18
                    食物名稱                                                 食材  \
0   【Angry Birds米披薩】 免烤箱                [白飯, 小熱狗, 海苔, 起士片黃，白, 乳酪絲, 番茄醬, 蘋果]   
1      Olivia♥蒜辣鯷魚蘆筍義大利麵  [義大利麵, 大蒜, 乾辣椒片, 油漬鯷魚, 蘆筍, 橄欖油, 鹽(燙煮義大利麵), 鹽和黑胡椒]   
2             香辣筍香鮪魚義大利麵  [香筍鮪魚罐頭, 蔥, 蒜頭, 辣椒, 高麗菜, 義大利麵1~2人份, 辣油, 塩, 黑胡椒...   
3            【烘培筆記】披薩盒子〞                         [厚片吐司, 甜椒, 熱狗, 番茄醬, 焗烤乳酪絲]   
4             香濃起司節瓜義大利麵  [義大利筆管麵, 橄欖油, 節瓜, 洋蔥, 芝思樂起司片, Parmigioano 帕馬森起...   
5               海鮮蕃茄義大利麵   [義大利麵條, 蛤蠣, 鮮蝦, 花枝, 番茄醬, 鹽, 羅勒葉(或是九層塔), 起司粉, 奶油]   
6     千變萬化的-披薩皮 -烘焙展食譜募集                         [高筋麵粉, 水, 橄欖油, 塩, 砂糖, 酵母粉]   
7             焗烤鮮蝦白醬義大利麵  [蝦仁, 綠花椰菜, 紅蘿蔔, 馬鈴薯, 玉米筍, 絞肉, 洋蔥, 鮮奶, 飲氧品, 義大利...   
8             水餃皮的變化-小披薩         [餃皮, 臘肉(魚肉, 培根等。), 番茄醬, 起司絲, 蔥花, 紅蘿蔔絲, 番茄]   
9        【靜靜小廚】剩菜剩飯變身米披薩  [煮熟的米飯, 雞蛋, 鹽, 胡椒粉, 黑胡椒豬柳(剛好冰箱剩這個，大家可視當天剩菜是什麼隨...   
10              海鮮義大利麵吐司  [未切片12兩吐司, 義大利直圓麵, 蝦子, 花枝圈, 蘆筍, 蒜頭, 起司片(切絲), 巴...   
11      輕鬆好料理~番茄肉醬蛤蠣義大利麵  [牛番茄, 蛤蠣, 絞肉, 大蒜, 洋蔥, 冰

18
                                                 食物名稱  \
0                                           零麵粉pizza皮   
1                                         十分鐘上桌❤️吐司比薩   
2                                             義式海鮮蕃茄麵   
3                                             義式菠菜起司餃   
4                                              童趣義大利麵   
5                                          簡單做拿坡里義大利麵   
6                                          番茄海鮮白醬義大利麵   
7                                           義式香料烤雞&薯條   
8                                              義大利肉醬麵   
9                                        ＜義式＞羊肉洋蔥番茄醬麵   
10                                           清炒蔬菜義大利麵   
11                                   ♥我的手作料理♥咖哩南瓜義大利麵   
12                     Miss Sia[做]自製比薩 Homemade Pizza   
13  無花果焦糖洋蔥披薩搭配藍黴起司 Fig Jam, Caramelized Onion & B...   
14                                         快速蝦仁蔬菜義大利麵   
15                                              義式炒魷魚   
16                          

18
                                        食物名稱  \
0                                   明太子醬烤鮭魚麵   
1         自己動手做比薩~Homemade Pizza Margherita!   
2                                   奶焗照燒雞肉披薩   
3                                      義式烤茄子   
4                                卡非南汁龍鱈焗義大利麵   
5                       [阿妮塔♥萬聖節] 熱狗木乃伊義大利麵。   
6                            金門高粱飄香料理-茄汁義大利麵   
7                              《剩飯系列》免烤箱~披薩餃   
8   辣味蕃茄肉丸義大利麵 Spicy Marinara Meatball Pasta   
9                             起司肉醬義大利麵(ㄧ鍋到底)   
10       Miss Sia[做] 義式沙拉米拼盤Italian Cold Cut   
11                    【吳双の愛妻料理】農家風味的豬肉蔬菜義大利麵   
12                     培根蛋奶麵 Pasta Carbonara   
13                                     pizza   
14     一鍋到底泰式酸辣義大利麵 One-pan Thai Style Pasta   
15                               不用烤箱的 PIZZA   
16                                青醬德式香腸義大利麵   
17                                涼拌鮪魚時蔬義大利麵   

                                                   食材  \
0            [義大利麵, 鮭魚一夜干, 

18
                                                 食物名稱  \
0                                  【伊莉莎白辣醬食譜】蕃茄辣醬義大利麵   
1                                            番茄起司義大利麵   
2                                    ♥憶柔蔬食♥金黃白醬蝴蝶義大利麵   
3                                              海鮮義大利麵   
4                                                隨手披薩   
5                                             女孩義式肉丸麵   
6                                          ~蒜香起士義大利麵~   
7                                     蒜片辣椒義大利麵。by克莉絲塔   
8   苦瓜鹹蛋義大利麵 Linguine with Bitter Melon & Salted D...   
9                                             自製夏威夷披薩   
10                                            火腿炒義大利麵   
11                                  超簡單~吐司批薩(黑胡椒熱狗口味)   
12                                           海鮮青醬義大利麵   
13                                         義大利麵ft.水波蛋   
14                                         香濃蕃茄肉丸義大利麵   
15                                         ~玉米鮪魚義大利麵~   
16                          

好像是最後一頁
